### Import bibliothèques

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [4]:
import os


def lister_fichiers(repertoire):
    return os.listdir(repertoire)

#contient les images dans lesquelles on souhaite faire la recherche
liste_images=lister_fichiers("IMAGES/Images")
liste_images.sort()

#imagettes contenant les caractéristiques à chercher dans les images
liste_avions=lister_fichiers('IMAGES/Imagettes/avions')
liste_drones=lister_fichiers('IMAGES/Imagettes/drones')
liste_helicopter=lister_fichiers('IMAGES/Imagettes/hélicoptères')
liste_missiles=lister_fichiers('IMAGES/Imagettes/missiles')
liste_oiseaux=lister_fichiers('IMAGES/Imagettes/oiseaux')


 ### Fonction de comparaison ORB :

In [116]:
def comparaisonORB (path1, path2):
    img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)

    if img1 is None:
        print(f"Impossible de lire l'image : {path1}")
        return 0
    
    img1 = cv2.resize(img1, (512,512))

    
    img2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

    if img2 is None:
        print(f"Impossible de lire l'image : {path2}")
        return 0

    img2 = cv2.resize(img2, (512,512))

    # Vérification des images chargées, on retourne un grand nombre pour ne pas fausser le résultat si l'image n'est pas chargée.
    if img1 is None:
        return 0
    if img2 is None:
        return 0

    orb = cv2.ORB_create()
    kp1, descripteursImg1 = orb.detectAndCompute(img1, None)
    kp2, descripteursImg2 = orb.detectAndCompute(img2, None)
    
    algoBF = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    paires_corresp = algoBF.knnMatch(descripteursImg1, descripteursImg2, k=2)

    Matches_tri = sorted(paires_corresp, key=lambda x:x[0].distance)
    
    # Application du test de ratio.
    matches_ratio = [x[0] for x in Matches_tri
            if len(x) > 1 and x[0].distance < 0.75 * x[1].distance]

    sumDistances = 0

    for match in matches_ratio:
        #for match in group :
            sumDistances += match.distance


    if len(matches_ratio) > 4:
    # Récupérer les coordonnées (x, y) pour chaque match
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches_ratio]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches_ratio]).reshape(-1, 1, 2)

        # Calcul de l'homographie avec RANSAC
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        # 'mask' est un array de la même longueur que matches_ratio
        # mask[i] = 1 => le match i est inlier
        # mask[i] = 0 => le match i est outlier

        inliers = [m for m, valid in zip(matches_ratio, mask) if valid]
        nb_inliers = len(inliers)
    else:
        nb_inliers = 0

    #score

    score=nb_inliers/min(len(kp1), len(kp2))

    return score


 ### Pourcentage de réussite avec ORB

In [117]:
nb_true=0
nb_false=0


for i in range(0,50):   # Parcours de toutes nos images a tester

    DicoComparaison = {"Avions" : [], "Drones" : [], "Hélicoptères" : [], "Missiles" : [], "Oiseaux" : []} # initialisation des doctionnaires vides
    pathUser = "IMAGES/Images/" + liste_images[i]

    for j in range(10): # Comparaison avec les imagettes
        pathAvions = 'IMAGES/Imagettes/avions/' + liste_avions[j]
        nb=comparaisonORB(pathUser, pathAvions)
        DicoComparaison["Avions"].append(nb)
        pathDrones = 'IMAGES/Imagettes/drones/' + liste_drones[j]
        nb=comparaisonORB(pathUser, pathDrones)
        DicoComparaison["Drones"].append(nb)
        pathHelico = 'IMAGES/Imagettes/hélicoptères/' + liste_helicopter[j]
        nb=comparaisonORB(pathUser, pathHelico)
        DicoComparaison["Hélicoptères"].append(nb)
        pathMissiles = 'IMAGES/Imagettes/missiles/' + liste_missiles[j]
        nb=comparaisonORB(pathUser, pathMissiles)
        DicoComparaison["Missiles"].append(nb)
        pathOiseaux = 'IMAGES/Imagettes/oiseaux/' + liste_oiseaux[j]
        nb=comparaisonORB(pathUser, pathOiseaux)
        DicoComparaison["Oiseaux"].append(nb)

    # Sommes des distances
    sumAvions = np.mean(DicoComparaison["Avions"])
    sumOiseaux = np.mean(DicoComparaison["Oiseaux"])
    sumDrones = np.mean(DicoComparaison["Drones"])
    sumHelicoptere = np.mean(DicoComparaison["Hélicoptères"])
    sumMissiles = np.mean(DicoComparaison["Missiles"])

    sums = {
        "sumAvions": sumAvions,
        "sumDrones": sumDrones,
        "sumMissiles": sumMissiles,
        "sumHelicoptere": sumHelicoptere,
        "sumOiseaux": sumOiseaux,
    }

    # Trouvez le nom de la variable avec la valeur sumimale (la plus grande étant celle avec le plus de bonnes correspondances)
    minVar = max(sums, key=sums.get)
    
    #test de conformité : étant donné que les images sont triées par noms, les 9 premières sont des avions, etc...
    if 0<=i<=9:
        if minVar == "sumAvions":
            nb_true+=1
        else:
            nb_false+=1

    elif 10<=i<=19:
        if minVar == "sumDrones":
            nb_true+=1
        else:
            nb_false+=1

    elif 20<=i<=29:
        if minVar == "sumHelicoptere":
            nb_true+=1
        else:
            nb_false+=1

    elif 30<=i<=39:
        if minVar == "sumMissiles":
            nb_true+=1
        else:
            nb_false+=1

    elif 40<=i<=49:
        if minVar == "sumOiseaux":
            nb_true+=1
        else:
            nb_false+=1
    print(minVar)
    print(nb_true)
    print(sumAvions, sumDrones, sumHelicoptere, sumMissiles, sumOiseaux)

print("L'algorithme ORB obtient",(nb_true/50)*100, "% de réussite." )

sumAvions
1
0.0 0.0 0.0 0.0 0.0
sumMissiles
1
0.0 0.0 0.0016 0.001951219512195122 0.0
sumDrones
1
0.0 0.0022123893805309734 0.0008849557522123894 0.0008908685968819598 0.0011061946902654867
sumAvions
2
0.0034364261168384883 0.001374570446735395 0.0017182130584192442 0.0 0.0024054982817869417
sumDrones
2
0.0008 0.0044 0.0016 0.0008 0.0
sumHelicoptere
2
0.0008 0.0016 0.0021999999999999997 0.0 0.002
sumAvions
3
0.0038000000000000004 0.0016 0.0016 0.0 0.0
sumMissiles
3
0.0 0.0 0.0017094017094017096 0.0019943019943019944 0.0
sumAvions
4
0.004 0.0008 0.0008 0.0008 0.0024000000000000002
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES/Images/avion9.jpg
Impossible de lire l'image : IMAGES

 ### Fonction de comparaison SIFT

In [26]:
def comparaisonSIFT (path1, path2):
    img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    
    img2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

     # Vérification des images chargées
    if img1 is None:
        return 0
    if img2 is None:
        return 0


    sift = cv2.SIFT_create()
    points1, descripteursImg1 = sift.detectAndCompute(img1, None)
    points2, descripteursImg2 = sift.detectAndCompute(img2, None)
    
    algoBF = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    paires_corresp = algoBF.knnMatch(descripteursImg1, descripteursImg2, k=5)

    Matches_tri = sorted(paires_corresp, key=lambda x:x[0].distance)

    Matches_tri = Matches_tri[:10]
    
    sumDistances = 0

    for group in Matches_tri:
        for match in group :
            sumDistances += match.distance

    return sumDistances

### Pourcentage de réussite avec l'algorithme SIFT

In [27]:
nb_true=0
nb_false=0


for i in range(0,50):   # Parcours de toutes nos images a tester

    DicoComparaison = {"Avions" : [], "Drones" : [], "Hélicoptères" : [], "Missiles" : [], "Oiseaux" : []} # initialisation des doctionnaires vides
    pathUser = "IMAGES/Images/" + liste_images[i]

    for j in range(10): # Comparaison avec les imagettes
        pathAvions = 'IMAGES/Imagettes/avions/' + liste_avions[j]
        nb=comparaisonSIFT(pathUser, pathAvions)
        DicoComparaison["Avions"].append(nb)
        pathDrones = 'IMAGES/Imagettes/drones/' + liste_drones[j]
        nb=comparaisonSIFT(pathUser, pathDrones)
        DicoComparaison["Drones"].append(nb)
        pathHelico = 'IMAGES/Imagettes/hélicoptères/' + liste_helicopter[j]
        nb=comparaisonSIFT(pathUser, pathHelico)
        DicoComparaison["Hélicoptères"].append(nb)
        pathMissiles = 'IMAGES/Imagettes/missiles/' + liste_missiles[j]
        nb=comparaisonSIFT(pathUser, pathMissiles)
        DicoComparaison["Missiles"].append(nb)
        pathOiseaux = 'IMAGES/Imagettes/oiseaux/' + liste_oiseaux[j]
        nb=comparaisonSIFT(pathUser, pathOiseaux)
        DicoComparaison["Oiseaux"].append(nb)

    # Sommes des distances
    sumAvions = np.sum(DicoComparaison["Avions"])
    sumOiseaux = np.sum(DicoComparaison["Oiseaux"])
    sumDrones = np.sum(DicoComparaison["Drones"])
    sumHelicoptere = np.sum(DicoComparaison["Hélicoptères"])
    sumMissiles = np.sum(DicoComparaison["Missiles"])

    sums = {
        "sumAvions": sumAvions,
        "sumDrones": sumDrones,
        "sumMissiles": sumMissiles,
        "sumHelicoptere": sumHelicoptere,
        "sumOiseaux": sumOiseaux,
    }

    # Trouvez le nom de la variable avec la valeur minimale (la plus petite étant censé être la plus ressemblante)
    minVar = min(sums, key=sums.get)
    minSum = sums[minVar]
    
    #test de conformité : étant donné que les images sont triées par noms, les 9 premières sont des avions, etc...
    if 0<=i<=9:
        if minVar == "sumAvions":
            nb_true+=1
        else:
            nb_false+=1

    elif 10<=i<=19:
        if minVar == "sumDrones":
            nb_true+=1
        else:
            nb_false+=1

    elif 20<=i<=29:
        if minVar == "sumHelicoptere":
            nb_true+=1
        else:
            nb_false+=1

    elif 30<=i<=39:
        if minVar == "sumMissiles":
            nb_true+=1
        else:
            nb_false+=1

    elif 40<=i<=49:
        if minVar == "sumOiseaux":
            nb_true+=1
        else:
            nb_false+=1
            
    print(minVar)
    print(i)
    print(nb_true)
    

print("L'algorithme SIFT obtient",round((nb_true/50)*100), "% de réussite en prenant en compte la distance comme critère." )

sumAvions
0
1
sumAvions
1
2
sumOiseaux
2
2
sumOiseaux
3
2
sumOiseaux
4
2
sumOiseaux
5
2
sumHelicoptere
6
2
sumOiseaux
7
2
sumOiseaux
8
2
sumAvions
9
3
sumHelicoptere
10
3
sumOiseaux
11
3
sumOiseaux
12
3
sumOiseaux
13
3
sumOiseaux
14
3
sumAvions
15
3
sumHelicoptere
16
3
sumOiseaux
17
3
sumOiseaux
18
3
sumOiseaux
19
3
sumOiseaux
20
3
sumOiseaux
21
3
sumOiseaux
22
3
sumOiseaux
23
3
sumOiseaux
24
3
sumOiseaux
25
3
sumAvions
26
3
sumOiseaux
27
3
sumAvions
28
3
sumOiseaux
29
3
sumOiseaux
30
3
sumAvions
31
3
sumOiseaux
32
3
sumOiseaux
33
3
sumOiseaux
34
3
sumOiseaux
35
3
sumOiseaux
36
3
sumOiseaux
37
3
sumOiseaux
38
3
sumAvions
39
3
sumOiseaux
40
4
sumAvions
41
4
sumAvions
42
4
sumHelicoptere
43
4
sumAvions
44
4
sumAvions
45
4
sumHelicoptere
46
4
sumAvions
47
4
sumOiseaux
48
5
sumOiseaux
49
6
L'algorithme SIFT obtient 12 % de réussite en prenant en compte la distance comme critère.
